In [1]:
import pygame
import random
import math
import numpy as np 

# 常量，屏幕宽高
WIDTH, HEIGHT = 1000, 1000

# 初始化操作
pygame.init()
pygame.mixer.init()
# 创建游戏窗口
screen = pygame.display.set_mode((WIDTH, HEIGHT))

# 设置游戏标题
pygame.display.set_caption('妹多局之卡坦岛')

pygame 2.3.0 (SDL 2.24.2, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
red, blue, yellow = [pygame.Color(color) for color in ['red', 'blue', 'yellow']]

In [3]:
# 添加系统时钟，用于设置帧的刷新
FPS = 10
clock = pygame.time.Clock()

In [4]:
class Text(pygame.sprite.Sprite):
    def __init__(self, text, center, size=20, color=blue, width=10, height=10):
        super().__init__()
    
        self.font = pygame.font.SysFont("Arial", size)
        self.text_surf = self.font.render(text, 1, color)
        self.image = pygame.Surface((width, height))
        W = self.text_surf.get_width()
        H = self.text_surf.get_height()
        self.rect = self.image.get_rect()
#         self.image.fill(color)
        self.image.blit(self.text_surf, center)
        
        
class Edge(pygame.sprite.Sprite):
    WIDTH = 5 
    def __init__(self, start_pos, end_pos, color=yellow):
        super().__init__()
        self.image = pygame.Surface([10, 50])
        self.image.set_colorkey(color)
        self.rect = self.image.get_rect(center=((start_pos[0] + end_pos[0])/2, (start_pos[-1]+end_pos[-1])/2))
        pygame.draw.line(screen, color, start_pos, end_pos, self.WIDTH)
    
    def update(self, color):
#         self.image.set_colorkey(color)
        colorImage = pygame.Surface(self.image.get_size()).convert_alpha()
        colorImage.fill(color)
        self.image.blit(colorImage, self.rect.center, special_flags = pygame.BLEND_RGBA_MULT)

In [5]:
# 定义class
class Vertex(pygame.sprite.Sprite):
    WIDTH, HEIGHT = 15, 15
    
    def __init__(self, x, y, color=red):
        super().__init__()
        self.center = (x, y)
        self.rect = pygame.rect.Rect(x, y, self.WIDTH, self.HEIGHT)
        self.image = pygame.Surface((self.WIDTH, self.HEIGHT))
#         self.image.fill(color)
        
        # 用颜色填充
        color_image = pygame.Surface(self.image.get_size()).convert_alpha()
        color_image.fill(color)
        self.image.blit(color_image, (x, y), special_flags = pygame.BLEND_RGBA_MULT)
    
    def update(self, color):
        colorImage = pygame.Surface(self.image.get_size()).convert_alpha()
        colorImage.fill(color)
        self.image.blit(colorImage, self.center, special_flags = pygame.BLEND_RGBA_MULT)

In [6]:
class Hexagon(pygame.sprite.Sprite):
    """hexagonal tile
    
    Attrs:
        no: rolling number
        res: resource type, options ['wool', 'lumber', 'wheat', 'ore', 'brick', 'jpeg']
        vertices: six vertices, where players can build settlements
        edges: six lines, where players can build road
    """
    RADIUS = 40 
    RESOURCE_TYPES = 'wool lumber wheat ore brick'.split(' ')
    RESOURCE_IMAGES = [pygame.image.load(f"./images/catan/{res}.jpeg") for res in RESOURCE_TYPES]
    TIPO2IMAGE = dict(zip(RESOURCE_TYPES, RESOURCE_IMAGES))
    G3 = math.sqrt(3)
    WIDTH, HEIGHT = RADIUS * 2 * G3 - 2 * Edge.WIDTH, 2 * RADIUS - 2 * Edge.WIDTH
    
    def __init__(self, resource_type, center_x, center_y, no):
        super().__init__()
        self.no = no
        self.center_x, self.center_y = center_x, center_y
        self.res = resource_type
        self.image = self.TIPO2IMAGE[self.res]
        self.image = pygame.transform.scale(self.image, (self.WIDTH, self.HEIGHT))
        self.rect = pygame.rect.Rect(center_x - self.WIDTH / 2, center_y - self.RADIUS, self.WIDTH, self.HEIGHT)

    def update(self):
        self.text = Text(str(self.no), center=(self.center_x, self.center_y-self.RADIUS * 1.1))
        angles = [i * math.pi / 3 + math.pi / 6 for i in range(6)]
        self.vertices = [
            Vertex(
                x=self.center_x + self.RADIUS * 2 * math.cos(angle),
                y=self.center_y + self.RADIUS * 2 * math.sin(angle),
                
            ) 
            for angle in angles
        ]
        
        self.edges = [
            Edge(start_pos=self.vertices[i].center, end_pos=self.vertices[(i+1) % 6].center)
            for i in range(6)
        ]

In [7]:
# 初始化精灵组
hex_group = pygame.sprite.Group()
ver_group = pygame.sprite.Group()
text_group = pygame.sprite.Group()
edge_group = pygame.sprite.Group()

# 定义人物
hexs = [
    Hexagon(
        no=random.randint(2, 12),
        center_x=Hexagon.G3 * Hexagon.RADIUS * (2*i+1),
        center_y=3 * Hexagon.RADIUS,
        resource_type=random.choice(Hexagon.RESOURCE_TYPES)
    ) for i in range(6)
]

for ele in hexs:
    hex_group.add(ele)
#     for vertex in ele.vertices:
#         ver_group.add(vertex)
#     for edge in ele.edges:
#         edge_group.add(edge)
#     text_group.add(ele.text)

In [8]:
# 保持游戏运行状态(游戏循环）
while True:
    # ===========游戏帧的刷新===========
    clock.tick(FPS)

    # 检测事件
    for event in pygame.event.get():
        # 检测关闭按钮被点击的事件
        if event.type == pygame.QUIT:
            # 退出
            pygame.quit()
            exit()
    
    screen.fill((100, 100, 100))
    for group in [hex_group]:
        group.update()
        for edge in hexs[0].edges:
            edge.update(red)

        group.draw(screen)
    
    pygame.display.update()

error: display Surface quit